Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = '../notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    # Note: wouldn't getting out of with remove save object?
    
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (500000, 28, 28) (500000,)
Validation set (29000, 28, 28) (29000,)
Test set (18000, 28, 28) (18000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return (dataset, labels)

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (500000, 28, 28, 1) (500000, 10)
Validation set (29000, 28, 28, 1) (29000, 10)
Test set (18000, 28, 28, 1) (18000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

Note: Initial configuration was slightly changed while experimenting.

In [8]:
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # layerX_weights are the kernels of the convolution
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        # debug:
        # print(shape)
        # exit()
        # shape[0] is batch size !
        # we reshape to feed the output of the convolution layer to the fully connected hidden layer.
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return(tf.matmul(hidden, layer4_weights) + layer4_biases)
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.658078
Minibatch accuracy: 4.7%
Validation accuracy: 13.2%
Minibatch loss at step 500: 1.114719
Minibatch accuracy: 71.9%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.433976
Minibatch accuracy: 84.4%
Validation accuracy: 84.2%
Minibatch loss at step 1500: 0.492276
Minibatch accuracy: 84.4%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.522514
Minibatch accuracy: 85.9%
Validation accuracy: 85.7%
Minibatch loss at step 2500: 0.371346
Minibatch accuracy: 85.9%
Validation accuracy: 86.5%
Minibatch loss at step 3000: 0.468736
Minibatch accuracy: 84.4%
Validation accuracy: 86.8%
Minibatch loss at step 3500: 0.586095
Minibatch accuracy: 76.6%
Validation accuracy: 87.1%
Minibatch loss at step 4000: 0.525313
Minibatch accuracy: 84.4%
Validation accuracy: 87.5%
Minibatch loss at step 4500: 0.485173
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 0.372806
Minibatch accuracy: 89.1%
Validation accuracy

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [24]:
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # layerX_weights are the kernels of the convolution
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # we need to change layer 3 size because of the addition of max_pool
    # 28//16 = 1 but we need 2 - hard code it for now!
    layer3_weights = tf.Variable(tf.truncated_normal(
        [(image_size // 16 +1) * (image_size // 16 +1) * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        # print(data.get_shape().as_list())
        # [64, 28, 28, 1]
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        # print(hidden.get_shape().as_list())
        # [64, 14, 14, 16]
        mpool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        # print(mpool.get_shape().as_list())
        # [64, 7, 7, 16]
        conv = tf.nn.conv2d(mpool, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        # print(hidden.get_shape().as_list())
        # [64, 4, 4, 16]
        mpool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = mpool.get_shape().as_list()
        # print(shape)
        # [64, 2, 2, 16]
        reshape = tf.reshape(mpool, [shape[0], shape[1] * shape[2] * shape[3]])
        # print(reshape.get_shape().as_list())
        # [64, 64]
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return(tf.matmul(hidden, layer4_weights) + layer4_biases)
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [26]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.515413
Minibatch accuracy: 6.2%
Validation accuracy: 14.6%
Minibatch loss at step 500: 1.154397
Minibatch accuracy: 67.2%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 0.464177
Minibatch accuracy: 89.1%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.450517
Minibatch accuracy: 82.8%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 0.523223
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 2500: 0.359377
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 0.428505
Minibatch accuracy: 89.1%
Validation accuracy: 86.4%
Minibatch loss at step 3500: 0.587077
Minibatch accuracy: 81.2%
Validation accuracy: 86.7%
Minibatch loss at step 4000: 0.582631
Minibatch accuracy: 82.8%
Validation accuracy: 87.1%
Minibatch loss at step 4500: 0.447280
Minibatch accuracy: 85.9%
Validation accuracy: 87.2%
Minibatch loss at step 5000: 0.308824
Minibatch accuracy: 90.6%
Validation accuracy

We got almost the same accuracy as with the previous effort. We likely didn't make efficient use of max pooling. Let's try again:

- We try by separating the convolutions in each dimension.

In [37]:
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # layerX_weights are the kernels of the convolution
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # we need to change layer 3 size because of the addition of max_pool
    # 28//16 = 1 but we need 2 - hard code it for now!
    layer3_weights = tf.Variable(tf.truncated_normal(
        [round(image_size / 8) * round(image_size / 8) * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        # print(data.get_shape().as_list())
        # [64, 28, 28, 1]
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        # print(hidden.get_shape().as_list())
        # [64, 14, 14, 16]
        mpool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 1, 2, 1], padding='SAME')
        # print(mpool.get_shape().as_list())
        # [64, 7, 7, 16]
        conv = tf.nn.conv2d(mpool, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        # print(hidden.get_shape().as_list())
        # [64, 4, 4, 16]
        mpool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 1, 1], padding='SAME')
        shape = mpool.get_shape().as_list()
        # print(shape)
        # [64, 2, 2, 16]
        reshape = tf.reshape(mpool, [shape[0], shape[1] * shape[2] * shape[3]])
        # print(reshape.get_shape().as_list())
        # [64, 64]
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return(tf.matmul(hidden, layer4_weights) + layer4_biases)
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [38]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.803274
Minibatch accuracy: 12.5%
Validation accuracy: 15.8%
Minibatch loss at step 500: 1.034481
Minibatch accuracy: 71.9%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 0.401738
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 0.479847
Minibatch accuracy: 89.1%
Validation accuracy: 86.0%
Minibatch loss at step 2000: 0.514692
Minibatch accuracy: 85.9%
Validation accuracy: 86.9%
Minibatch loss at step 2500: 0.348562
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 3000: 0.383884
Minibatch accuracy: 89.1%
Validation accuracy: 87.7%
Minibatch loss at step 3500: 0.586743
Minibatch accuracy: 81.2%
Validation accuracy: 87.8%
Minibatch loss at step 4000: 0.484448
Minibatch accuracy: 85.9%
Validation accuracy: 87.8%
Minibatch loss at step 4500: 0.499369
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 5000: 0.292312
Minibatch accuracy: 89.1%
Validation accurac

We got slightly better results. Seems like two max pools after each convolution on such a small image size was excessive.

 - Let's retry with only max pool

In [39]:
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # layerX_weights are the kernels of the convolution
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # we need to change layer 3 size because of the addition of max_pool
    # 28//16 = 1 but we need 2 - hard code it for now!
    layer3_weights = tf.Variable(tf.truncated_normal(
        [round(image_size / 8) * round(image_size / 8) * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        # print(data.get_shape().as_list())
        # [64, 28, 28, 1]
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        # print(hidden.get_shape().as_list())
        # [64, 14, 14, 16]
        mpool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        # print(mpool.get_shape().as_list())
        # [64, 7, 7, 16]
        conv = tf.nn.conv2d(mpool, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        # print(shape)
        # [64, 4, 4, 16]
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        # print(reshape.get_shape().as_list())
        # [64, 256]
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return(tf.matmul(hidden, layer4_weights) + layer4_biases)
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [40]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.279596
Minibatch accuracy: 6.2%
Validation accuracy: 9.5%
Minibatch loss at step 500: 1.164462
Minibatch accuracy: 68.8%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 0.392067
Minibatch accuracy: 87.5%
Validation accuracy: 84.6%
Minibatch loss at step 1500: 0.433566
Minibatch accuracy: 87.5%
Validation accuracy: 85.4%
Minibatch loss at step 2000: 0.510297
Minibatch accuracy: 84.4%
Validation accuracy: 86.1%
Minibatch loss at step 2500: 0.376562
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 3000: 0.400672
Minibatch accuracy: 84.4%
Validation accuracy: 87.3%
Minibatch loss at step 3500: 0.577692
Minibatch accuracy: 79.7%
Validation accuracy: 87.5%
Minibatch loss at step 4000: 0.471687
Minibatch accuracy: 84.4%
Validation accuracy: 87.6%
Minibatch loss at step 4500: 0.446904
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 0.272240
Minibatch accuracy: 92.2%
Validation accuracy:

Looks like putting the max_pooling operations together offers almost, but not exactly the same, performance as doing them separately.

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---